In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Basics of GBM - Gradient Descent, Boosting & GBDT
* What is Gradient Descent
![GBDT](https://miro.medium.com/max/700/1*3yAe8DsD7aZ4AucCm-7L0Q.png)

*It is an optimization technique to reduce the loss function by following the slope through a fixed step size*

### What is a Boosting?
*Boosting is sequential ensembling technique where hard to classify instances are given more weights*
![boosting](https://miro.medium.com/max/422/1*zFOpJUSU_zgg0FCPBOSXvg.png)

### Important points

1. Final model would be weighted average of n weak learners
2. Subsequent learners give more emphasis on undertrained / wrongly classified samples
3. Larger the size of dot larger is the weight assigned to it. Do note that all models are sequential

### GBDT

*In each iteration GBDT learns the decision tree by fitting the residual errors (errors upto the current iteration). This means every subsequent learner tries to learn the difference between actual output and weighted sum of predictions until an iteration before. The errors are minimised using the gradient method.*

![split point](https://miro.medium.com/max/700/0*o0E5qzKxkbfzd6hv)

### How to create the optimum split points?

1. Pre-sorted algorithm - enumerates all possible split points on pre-sorted values
2. Histogram based algorithm - continuous features into descrete bins to construct feature histograms during training

![split](https://miro.medium.com/max/456/1*qJOh7NqBlxPGe3xsiq4hoA.png)

*Both LightGBM and xgboost utilise histogram based split finding in contrast to sklearn which uses GBM ( One of the reasons why it is slow).*

## LGBM 알고리즘의 특성

### 1) down sampling data & Using GOSS (Gradient Based One Side Sampling)

* instances with small gradients are well trained (small training error)
* instances with large gradients are under trained

*GOSS retains instances with large gradients while performing random sampling on instances with small gradients*

### Intuitive steps for GOSS calculation

1. Sort the instances according to absolute gradients in a descending order
<br>

2. Select the top a * 100% instances. [ Under trained / large gradients ]
<br>

3. Randomly samples b * 100% instances from the rest of the data. This will reduce the contribution of well trained examples by a factor of b ( b < 1 )
<br>

4. Without point 3 count of samples having small gradients would be 1-a ( currently it is b ). In order to maintain the original distribution LightGBM amplifies the contribution of samples having small gradients by a constant (1-a)/b to put more focus on the under-trained instances. This puts more focus on the under trained instances without changing the data distribution by much.

### 2) EFB (Exclusive Feature Bundling)

희소한 변수 공간의 특성에 따라 배타적인 변수들을 하나의 변수로 묶을 수 있다. 그리고 이를 배타적 변수 묶음(Exclusive Feature Bundle)이라고 부른다. 정교하게 디자인된 변수 탐색 알고리즘을 통해, 각각의 변수들로 했던 것과 마찬가지로 변수 묶음들로부터도 동일한 변수 히스토그램들을 생성할 수 있게 된다.

![EFB](https://miro.medium.com/max/432/1*SigQkx_yJFh_5ZVmJvCpxQ.png)

가장 중요한 것은 변수 묶음들로부터 원래(original) 변수들의 값을 식별할 수 있어야 한다는 것이다. Histogram-based 알고리즘은 변수의 연속적인 값 대신 이산적인 구간(bin)을 저장하므로, 배타적 변수들을 각각 다른 구간에 두어 변수 묶음을 구성할 수 있다. 이는 변수의 원래 값에 offset을 더하는 것으로 이루어 질 수 있다.

![pic](https://greeksharifa.github.io/public/img/Machine_Learning/2019-12-09-Light%20GBM/04.JPG)

### 3) leaf_based decision tree
*리프 노드의 정보 이득을 우선하여 트리를 분할하는 원리로 동작한다*
![lgbm](https://miro.medium.com/max/1400/1*mKkwlQF25Rq1ilne5UiEXA.png)

1) 장점
* 트리의 최적 분할 포인트를 빠르게 수행할 수 있다 (모든 경우의 수를 고려하기 보다는, 최대 정보을 기반)
* 불순도의 최대 이득을 추구할 수 있다 -> 정확하게 분류할 가능성이 높다

2) 단점
* 불균형 트리 구조로 인해, 특정 피처에 기반에 과대적합 될 우려가 있다
* 작은 데이터 셋의 경우, 일반적으로 과대적합 가능성이 있다 (10000건 이하)

## LGBM 파라미터

### Basic parameters
1. max_depth : 결정트리의 최대 깊이
2. min_data_in_leaf : 리프 노드가 가지고 있는 최소한의 레코드 수 (이것 보다 적으면 분할 하지 않는다)
3. feature_fraction : iteration 수 만큼 반복하며 결정 트리를 만들 때, 사용할 feature 비율 (0.8 -> 80%)
4. early_stopping_round : validation 데이터 셋이 필요하다, 지나친 iteration을 방지하는 데 유용
5. lambda : regularization (규제정도 일반적으로 0~1사이 값을 지정)
6. min_gain_to_split : 트리가 분할하기 위해 필요한 최소한의 gain
7. max_cat_group : 카테고리 수가 클 때, 과적합을 방지하는 분기 포인트를 찾는다 (기본값은 64)

### Core parameters
1. application (regression - 회귀분석, binary - 이진 분류, multiclass - 다중 분류)
2. boosting (gdbt, rf, dart, goss)
3. n_estimations or num_boost_round (기본값 100)
4. learning_rate (추정값을 업데이트 하는 변화의 크기 값 - ex) 0.1, 0.001, 0.003 etc)
5. num_leaves (전체 트리의 리프 노드의 개수, 기본값은 31)

### IO parameters (input & output)

1. max_bin (feature 값의 최대 bin 수)
2. categorical_feature (범주형 feature의 인덱스를 지정)

## LGBM 파라미터 튜닝

### 모델의 정확도를 향상시키기 위해서
* num_leaves : Tree 모델의 복잡성을 컨트롤하는 주요 파라미터입니다. num_leaves 값은 2 ^ (max_depth) 값보다 적거나 같아야 합니다.

* min_data_in_leaf : 큰 값으로 세팅하는 것은 Tree가 너무 깊게 확장되는 것을 막을 수 있지만 under-fitting 언더 피팅이 발생할 수도 있습니다. 관행적으로, 수백 또는 수천 개로 정하는 것이 큰 데이터 세트에 충분합니다.


* max_depth : Tree 깊이를 명확하게 제한하기 위해 max_depth 값을 설정할 수도 있습니다.

### 더 빠른 속도를 위하여
* bagging_fraction & bagging_freq을 설정하여 bagging을 조절한다
* feature_fraction을 설정
* save_binary를 통해 데이터 로딩 속도를 줄여라
* parallel learing 병렬 학습을 적용

### 더 나은 정확도를 위해

* 큰 max_bin 값을 사용하십시오 (아마 속도는 느려질 수 있습니다)
* 작은 learning_rate 값을 큰 num_iterations 값과 함께 사용하십시오
* 큰 num_leaves 값을 사용하십시오 (아마 과적합을 유발할 수도 있습니다)
* 더 큰 트레이닝 데이터를 사용하십시오
* dart 를 사용하십시오
* 범주형 feature를 사용하십시오

### 과적합을 해결하기 위해
* 작은 max_bin 값을 사용하십시오
* 작은 num_leaves 값을 사용하십시오
* min_data_in_leaf 와 min_sum_hessian_in_leaf 파라미터를 사용하십시오
* bagging_fraction 과 bagging_freq 을 사용하여 bagging 을 적용하십시오
* feature_fraction을 세팅하여 feature sub-sampling을 하십시오
* lambda_l1, lambda_l2 그리고 min_gain_to_split 파라미터를 이용해 regularization (정규화) 를 적용하십시오
* max_depth 를 설정해 Deep Tree 가 만들어지는 것을 방지하십시오

## 핵심 파라미터 튜닝에 대한 가이드 라인

n_estimators 파라미터는 반복 수행하는 트리의 개수를 의미한다. 너무 크게 지정하면 학습 시간이 오래 걸리고 과적합이 발생할 수 있으니, 파라미터 튜닝 시에는 크지 않은 숫자로 지정하는 것이 좋다. 

num_leaves 파라미터는 하나의 트리가 가질 수 있는 최대 리프의 개수인데, 이 개수를 높이면 정확도는 높아지지만 트리의 깊이가 커져 모델의 복잡도가 증가한다는 점에 유의해야 한다.

max_depth 파라미터는 트리의 최대 깊이를 의미하는데, 위에서 설명한 num_leaves 파라미터와 중요한 관계를 지닌다. 과적합을 방지하기 위해 num_leaves는 2^(max_depth)보다 작아야 한다. 예를 들어 max_depth가 7이기 때문에, 2^(max_depth)=98이 되는데, 이 때 num_leaves를 이보다 작은 70~80 정도로 설정하는 것이 낫다.

min_child_samples 파라미터는 최종 결정 클래스인 Leaf Node가 되기 위해서 최소한으로 필요한 데이터 개체의 수를 의미하며, 과적합을 제어하는 파라미터이다. 이 파라미터의 최적값은 훈련 데이터의 개수와 num_leaves에 의해 결정된다. 너무 큰 숫자로 설정하면 under-fitting이 일어날 수 있으며, 아주 큰 데이터셋이라면 적어도 수백~수천 정도로 가정하는 것이 편리하다.

sub_sample 파라미터는 과적합을 제어하기 위해 데이터를 샘플링하는 비율을 의미한다.

지금까지 설명한 num_leaves, max_depth, min_child_samples, sub_sample 파라미터가 Light GBM 파라미터 튜닝에 있어서 가장 중요한 파라미터들이다. 이들은 하나씩 튜닝할 수도 있고, 한 번에 튜닝할 수도 있다. 학습 데이터의 성격과 여유 시간에 따라 선택해야 한다. 이들에 대한 최적값을 어느 정도 확보했다면, 다음 단계로 넘어가도 좋다.

colsample_bytree 파라미터는 개별 트리를 학습할 때마다 무작위로 선택하는 피쳐의 비율을 제어한다. reg_alpha는 L1 규제를, reg_lambda는 L2 규제를 의미한다. 이들은 과적합을 제어하기에 좋은 옵션들이다.

learning_rate은 후반부에 건드리는 것이 좋은데, 초반부터 너무 작은 학습률을 지정하면 효율이 크게 떨어질 수 있기 때문이다. 정교한 결과를 위해, 마지막 순간에 더욱 좋은 결과를 도출하기 위해 영혼까지 끌어모으고 싶다면, learning_rate는 낮추고 num_estimators는 크게 하여 최상의 결과를 내보도록 하자.

## 참고 웹페이지
* https://towardsdatascience.com/what-makes-lightgbm-lightning-fast-a27cf0d9785e
* https://nurilee.com/2020/04/03/lightgbm-definition-parameter-tuning/
* https://greeksharifa.github.io/machine_learning/2019/12/09/Light-GBM/
* https://lightgbm.readthedocs.io/en/latest/Features.html#references
* https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc